# (강의) 이미지캡셔닝

신록예찬  
2024-10-10

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Image captioning

Image captioning is the task of predicting a caption for a given image.
Common real world applications of it include aiding visually impaired
people that can help them navigate through different situations.
Therefore, image captioning helps to improve content accessibility for
people by describing images to them.

This guide will show you how to:

-   Fine-tune an image captioning model.
-   Use the fine-tuned model for inference.

Before you begin, make sure you have all the necessary libraries
installed:

``` bash
pip install transformers datasets evaluate -q
pip install jiwer -q
```

We encourage you to log in to your Hugging Face account so you can
upload and share your model with the community. When prompted, enter
your token to log in:

In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

## Load the Pokémon BLIP captions dataset

Use the 🤗 Dataset library to load a dataset that consists of
{image-caption} pairs. To create your own image captioning dataset in
PyTorch, you can follow [this
notebook](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/GIT/Fine_tune_GIT_on_an_image_captioning_dataset.ipynb).

In [1]:
from datasets import load_dataset

# Conceptual Captions 데이터셋 불러오기
ds = load_dataset("conceptual_captions", split="train")
ds

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

Dataset({
    features: ['image_url', 'caption'],
    num_rows: 3318333
})

In [5]:
ds['caption']

['a very typical bus station',
 'sierra looked stunning in this top and this skirt while performing with person at their former university',
 'young confused girl standing in front of a wardrobe',
 'interior design of modern living room with fireplace in a new house',
 'cybernetic scene isolated on white background .',
 'gangsta rap artist attends sports team vs playoff game in the borough .',
 'the jetty : different types of plants to establish a variety of ecosystems .',
 'traditional ornamental floral paisley bandanna .',
 '# of the sports team skates against sports team during their game .',
 'by geographical feature category or in the city - a dome for every environment',
 'a flight was traveling when the animal got free on tuesday night',
 'even though agricultural conditions are not ideal for growing tobacco , there is indigenous production .',
 'us state speaks during a demonstration thursday .',
 'actor arrives for the premiere of the film',
 'celebrities start decorating for 

In [14]:
ds

Dataset({
    features: ['image_url', 'caption'],
    num_rows: 3318333
})

In [16]:
ds = ds.train_test_split(test_size=0.1)
train_ds = ds["train"]
test_ds = ds["test"]

Let’s visualize a couple of samples from the training set.

In [22]:
train_ds

{'image_url': 'http://l7.alamy.com/zooms/a95a95341dcb4ebd810d5a30503bb98a/fisherman-sell-seafood-at-the-fish-market-fg4p25.jpg',
 'caption': 'sell seafood at the fish market'}

In [19]:
from textwrap import wrap
import matplotlib.pyplot as plt
import numpy as np


def plot_images(images, captions):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        caption = captions[i]
        caption = "\n".join(wrap(caption, 12))
        plt.title(caption)
        plt.imshow(images[i])
        plt.axis("off")


sample_images_to_visualize = [np.array(train_ds[i]["image"]) for i in range(5)]
sample_captions = [train_ds[i]["text"] for i in range(5)]
plot_images(sample_images_to_visualize, sample_captions)

    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/sample_training_images_image_cap.png" alt="Sample training images"/>

## Preprocess the dataset

Since the dataset has two modalities (image and text), the
pre-processing pipeline will preprocess images and the captions.

To do so, load the processor class associated with the model you are
about to fine-tune.

In [ ]:
from transformers import AutoProcessor

checkpoint = "microsoft/git-base"
processor = AutoProcessor.from_pretrained(checkpoint)

The processor will internally pre-process the image (which includes
resizing, and pixel scaling) and tokenize the caption.

In [ ]:
def transforms(example_batch):
    images = [x for x in example_batch["image"]]
    captions = [x for x in example_batch["text"]]
    inputs = processor(images=images, text=captions, padding="max_length")
    inputs.update({"labels": inputs["input_ids"]})
    return inputs


train_ds.set_transform(transforms)
test_ds.set_transform(transforms)

With the dataset ready, you can now set up the model for fine-tuning.

## Load a base model

Load the
[“microsoft/git-base”](https://huggingface.co/microsoft/git-base) into a
[`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM)
object.

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(checkpoint)

## Evaluate

Image captioning models are typically evaluated with the [Rouge
Score](https://huggingface.co/spaces/evaluate-metric/rouge) or [Word
Error Rate](https://huggingface.co/spaces/evaluate-metric/wer). For this
guide, you will use the Word Error Rate (WER).

We use the 🤗 Evaluate library to do so. For potential limitations and
other gotchas of the WER, refer to [this
guide](https://huggingface.co/spaces/evaluate-metric/wer).

In [ ]:
from evaluate import load
import torch

wer = load("wer")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predicted = logits.argmax(-1)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions = processor.batch_decode(predicted, skip_special_tokens=True)
    wer_score = wer.compute(predictions=decoded_predictions, references=decoded_labels)
    return {"wer_score": wer_score}

## Train!

Now, you are ready to start fine-tuning the model. You will use the 🤗
[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)
for this.

First, define the training arguments using
[TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments).

In [ ]:
from transformers import TrainingArguments, Trainer

model_name = checkpoint.split("/")[1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-pokemon",
    learning_rate=5e-5,
    num_train_epochs=50,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    logging_steps=50,
    remove_unused_columns=False,
    push_to_hub=True,
    label_names=["labels"],
    load_best_model_at_end=True,
)

Then pass them along with the datasets and the model to 🤗 Trainer.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

To start training, simply call
[train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train)
on the
[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)
object.

In [ ]:
trainer.train()

You should see the training loss drop smoothly as training progresses.

Once training is completed, share your model to the Hub with the
[push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub)
method so everyone can use your model:

In [ ]:
trainer.push_to_hub()

## Inference

Take a sample image from `test_ds` to test the model.

In [ ]:
from PIL import Image
import requests

url = "https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/pokemon.png"
image = Image.open(requests.get(url, stream=True).raw)
image

    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/test_image_image_cap.png" alt="Test image"/>

Prepare image for the model.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs = processor(images=image, return_tensors="pt").to(device)
pixel_values = inputs.pixel_values

Call `generate` and decode the predictions.

In [ ]:
generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)

``` bash
a drawing of a pink and blue pokemon
```

Looks like the fine-tuned model generated a pretty good caption!